In [1]:
%pwd

'/home/jatin/Projects/customer_churn_prediction/resarch'

In [2]:
import os
os.chdir("../")
%pwd

'/home/jatin/Projects/customer_churn_prediction'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelPredictionConfig:
    """
    Storing configuration related to the model evaluation.
    """
    root_dir: Path
    encoder_file: Path
    status_file: Path
    model_path: Path
    schema: dict


In [4]:
from customer_churn_prediction.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH, SCHEMA_FILE_PATH 
from customer_churn_prediction.utils.common import create_directory, read_yaml


class ConfigurationManager:
    """
    Handles loading and managing configuration, 
    parameters and schema for the project.
    """
    def __init__(
            self,
            config_path=CONFIG_FILE_PATH,
            schema_path=SCHEMA_FILE_PATH,
            params_path=PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_path)
        self.schema = read_yaml(schema_path)
        self.params = read_yaml(params_path)

        create_directory([self.config.artifacts_root])

    
    def get_prediction_config(self)->ModelPredictionConfig:
        """
        Return the Model Predictor config
        """
        transformation_config = self.config.data_transformation
        config = self.config.model_prediction
        schema = self.schema.COLUMNS

        create_directory([config.root_dir])

        model_predictor_config = ModelPredictionConfig(
            root_dir = config.root_dir,
            encoder_file = transformation_config.encoder_file,
            status_file = config.status_file,
            model_path = config.model_path,
            schema = schema
        )
        return model_predictor_config
    
    



In [5]:
import pandas as pd
import pickle
import joblib


from customer_churn_prediction import logger

class ModelPrediction:
    def __init__(self, config: ModelPredictionConfig):
        self.config = config

    def validate_data(self,data: pd.DataFrame):
        try:
            data_columns = data.dtypes.astype(str).to_dict()
            schema = self.config.schema
            msg = ""
            valiation_status = True
            for column_name,datatype in schema.items():
                if data_columns.get(column_name):
                    valiation_status = True if valiation_status else False
                    if data_columns.get(column_name) == datatype:
                        valiation_status = True if valiation_status else False
                        msg += f"{column_name} validated along with datatype"
                    else:
                        valiation_status = False
                        msg += f"{column_name} validated without datatype"
                else:
                    valiation_status = False
                    msg += f"{column_name} not validated"
                msg += "\n"
            with open(self.config.status_file,'w+') as f:
                f.write(f"Validation status: {valiation_status}")
            logger.info(f"Model Prediction stage data validation: \n {msg}")
        except Exception:
            logger.exception(f"Exception occured while validating the columns")
            return "Something went wrong"
        else:
            return msg

    def pre_process_data(self, data: pd.DataFrame):
        msg, is_data_processed, relevant_data = '', False, pd.DataFrame()
        
        status = False
        with open(self.config.status_file,'r+') as f:
            status_file_data = f.read()
            if status_file_data:
                status = status_file_data.rsplit(" ",1)
        if status:
            with open(self.config.encoder_file,"rb") as f:
                encoders = pickle.load(f)
                print(f"============================={[encoders]}")
                for col, encoder in encoders.items():
                    if col in data.columns:
                        data[col] = encoder.transform(data[col])
            relevant_data = data[list(self.config.schema.keys())]
            msg = "Data processed successfully"
            is_data_processed = True
        else:
            msg = "Data columns are not validated"
            is_data_processed = False
        return msg, is_data_processed, relevant_data 

    def predict(self, data: pd.DataFrame):
        status, msg = True, ''          
        validation_msg = self.validate_data(data)
        processing_msg, is_data_processed, relevant_data = self.pre_process_data(data)
        if is_data_processed:
            model = joblib.load(Path(self.config.model_path))
            prediction = model.predict(relevant_data)
            return status, prediction, msg
        else:
            status = False
            msg = processing_msg
            prediction = False
        return status, prediction, msg
            

In [6]:
data = {
    "gender": "Female",
    "SeniorCitizen": 0,
    "Partner": "Yes",
    "Dependents": "No",
    "tenure": 1,
    "PhoneService": "No",
    "MultipleLines": "No phone service",
    "InternetService": "DSL",
    "OnlineSecurity": "No",
    "OnlineBackup": "Yes",
    "DeviceProtection": "No",
    "TechSupport": "No",
    "StreamingTV": "No",
    "StreamingMovies": "No",
    "Contract": "Month-to-month",
    "PaperlessBilling": "Yes",
    "PaymentMethod": "Electronic check",
    "MonthlyCharges": 29.85,
    "TotalCharges": '29.85',
    "Churn":'Yes'
}

data = pd.DataFrame([data])


In [7]:
try:
    config = ConfigurationManager()
    data_prediction_config = config.get_prediction_config()
    model_prediction = ModelPrediction(data_prediction_config)
    status, prediction, msg = model_prediction.predict(data)
    # print(f"============================={[status, prediction, msg]}")
except Exception:
    logger.exception(
        f"Exception occured while executing the model evaluation pipeline")
    raise

[2026-01-03 13:12:10,508]:INFO:common.py:Yaml file: config/config.yaml is loaded successfully
[2026-01-03 13:12:10,515]:INFO:common.py:Yaml file: schema.yaml is loaded successfully
[2026-01-03 13:12:10,522]:INFO:common.py:Yaml file: params.yaml is loaded successfully
[2026-01-03 13:12:10,524]:INFO:common.py:Directory created at: artifacts
[2026-01-03 13:12:10,525]:INFO:common.py:Directory created at: artifacts/model_prediction
[2026-01-03 13:12:10,528]:INFO:2150127475.py:Model Prediction stage data validation: 
 gender validated along with datatype
SeniorCitizen validated along with datatype
Partner validated along with datatype
Dependents validated along with datatype
tenure validated along with datatype
PhoneService validated along with datatype
MultipleLines validated along with datatype
InternetService validated along with datatype
OnlineSecurity validated along with datatype
OnlineBackup validated along with datatype
DeviceProtection validated along with datatype
TechSupport valid

FileNotFoundError: [Errno 2] No such file or directory: 'artifacts/model_trainer/model.joblib'